In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(37) # 使得每次运行得到的随机数都一样

In [2]:
# 准备数据集
data_path='E:\PyProjects\DataSet\FireAI/data_multivar_2_class.txt'
df=pd.read_csv(data_path,header=None)
# print(df.head()) # 没有问题
print(df.info()) # 查看数据信息，确保没有错误
dataset_X,dataset_y=df.iloc[:,:-1],df.iloc[:,-1]
# print(dataset_X.head())
# print(dataset_X.info())
# print(dataset_y.head()) # 检查没问题
dataset_X=dataset_X.values
dataset_y=dataset_y.values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
0    300 non-null float64
1    300 non-null float64
2    300 non-null int64
dtypes: float64(2), int64(1)
memory usage: 7.1 KB
None


In [3]:
# 将整个数据集划分为train set和test set
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y=train_test_split(
    dataset_X,dataset_y,test_size=0.25,random_state=42)

In [7]:
from sklearn import svm, grid_search, cross_validation
from sklearn.metrics import classification_report

parameter_grid = [  {'kernel': ['linear'], 'C': [1, 10, 50, 600]}, # 需要优化的参数及其候选值
                    {'kernel': ['poly'], 'degree': [2, 3]},
                    {'kernel': ['rbf'], 'gamma': [0.01, 0.001], 'C': [1, 10, 50, 600]},
                 ]

metrics = ['precision', 'recall_weighted'] # 评价指标好坏的标准

for metric in metrics:
    print("Searching optimal hyperparameters for: {}".format(metric))

    classifier = grid_search.GridSearchCV(svm.SVC(C=1), 
            parameter_grid, cv=5, scoring=metric)
    classifier.fit(train_X, train_y)

    print("\nScores across the parameter grid:") 
    for params, avg_score, _ in classifier.grid_scores_:  # 打印出该参数下的模型得分
        print('{}: avg_scores: {}'.format(params,round(avg_score,3)))

    print("\nHighest scoring parameter set: {}".format(classifier.best_params_))

    y_pred =classifier.predict(test_X) # 此处自动调用最佳参数？？
    print("\nFull performance report:\n {}".format(classification_report(test_y,y_pred)))


Searching optimal hyperparameters for: precision


e:\programsoft\python\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
e:\programsoft\python\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
e:\programsoft\python\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
e:\programsoft\python\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
e:\programsoft\python\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricW


Scores across the parameter grid:
{'C': 1, 'kernel': 'linear'}: avg_scores: 0.809
{'C': 10, 'kernel': 'linear'}: avg_scores: 0.809
{'C': 50, 'kernel': 'linear'}: avg_scores: 0.809
{'C': 600, 'kernel': 'linear'}: avg_scores: 0.809
{'degree': 2, 'kernel': 'poly'}: avg_scores: 0.859
{'degree': 3, 'kernel': 'poly'}: avg_scores: 0.852
{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}: avg_scores: 1.0
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}: avg_scores: 0.0
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}: avg_scores: 0.968
{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}: avg_scores: 0.855
{'C': 50, 'gamma': 0.01, 'kernel': 'rbf'}: avg_scores: 0.946
{'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}: avg_scores: 0.975
{'C': 600, 'gamma': 0.01, 'kernel': 'rbf'}: avg_scores: 0.948
{'C': 600, 'gamma': 0.001, 'kernel': 'rbf'}: avg_scores: 0.968

Highest scoring parameter set: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

Full performance report:
              precision    recall  f1-score   support

          0       0.75   

In [12]:
best_classifier=svm.SVC(C=600,gamma=0.01,kernel='rbf') # 上面的full performance report的确使用的是最佳参数组合
best_classifier.fit(train_X, train_y)
y_pred =best_classifier.predict(test_X)
print("\nFull performance report:\n {}".format(classification_report(test_y,y_pred)))


Full performance report:
              precision    recall  f1-score   support

          0       1.00      0.92      0.96        36
          1       0.93      1.00      0.96        39

avg / total       0.96      0.96      0.96        75



In [13]:
y_pred_train=best_classifier.predict(train_X)
print("\nFull performance report on trainset:\n {}".format(classification_report(train_y,y_pred_train)))


Full performance report on trainset:
              precision    recall  f1-score   support

          0       0.94      0.95      0.94       114
          1       0.95      0.94      0.94       111

avg / total       0.94      0.94      0.94       225

